In [2]:
import os
import sys

import numpy as np

sys.path.append(os.path.abspath(os.path.join('../BayesFlow')))

from bayesflow.trainers import MetaTrainer, ParameterEstimationTrainer
from bayesflow.networks import SequenceNet, EvidentialNetwork, InvertibleNetwork, InvariantNetwork
from bayesflow.amortizers import MetaAmortizer, SingleModelAmortizer
from bayesflow.models import GenerativeModel
from bayesflow.diagnostics import true_vs_estimated

from generative_models import batch_generate_levy_lca, generate_levy_lca

from meta_aux_classes import *

%load_ext Cython
%load_ext wurlitzer

In [3]:
def model_prior(batch_size, n_models, p_vals=None):
    """
    Samples from the models' prior batch size times and converts to one-hot.
    Assumes equal model priors.
    ----------
    
    Arguments:
    batch_size : int  -- the number of samples to draw from the prior
    ----------
    
    Returns:
    m_true : np.ndarray of shape (batch_size, theta_dim) -- the samples batch of parameters
    """
    
    # Equal priors, if nothign specified
    if p_vals is None:
        p_vals = [1/n_models] * n_models
    m_idx = np.random.choice(n_models, size=batch_size, p=p_vals).astype(np.int32)
    return m_idx

In [4]:
class LCAPrior:
    def __init__(self, n_conditions=2, n_accumulators=2):
        self.n_conditions = n_conditions
        self.n_accumulators = n_accumulators
    
    def __call__(self, batch_size):
        p = np.random.uniform(0.10, 2.00, size=(batch_size, self.n_conditions * self.n_accumulators)).astype('double')
        a = np.random.uniform(0.05, 0.80, size=batch_size).astype('double')
        beta = np.random.uniform(0.01, 0.99, size=batch_size).astype('double')
        kappa = np.random.uniform(0.01, 0.99, size=batch_size).astype('double')
        ndt = np.random.uniform(0.1, 0.4, size=batch_size).astype('double')
        # alpha = np.random.uniform(1.0, 2.0, size=batch_size).astype('double')
        
        params = np.column_stack((p, a, beta, kappa, ndt))
        return params
    

class LCASimulator:
    def __init__(self, n_conditions=2, n_accumulators=2):
        self.n_conditions = n_conditions
        self.n_accumulators = n_accumulators
    
    @staticmethod
    def partition(n, c):
        """Partitions the integer n into c equally sized chunks and returns an c-lengthed array of chunk sizes."""
        return [n // c + (1 if x < n % c else 0)
                      for x in range (c)]
        
    def __call__(self, params, n_obs):
        n_sim = params.shape[0]
        
        # total number of drift rates = n_cond * n_acc
        n_p = self.n_conditions * self.n_accumulators
        
        # Split n_obs equally between conditions 
        n_obs_cond = self.partition(n_obs, self.n_conditions)

        # Shared parameters
        a, beta, kappa, ndt = tuple(params.T[n_p:])
        alpha = np.full_like(a, 2.0, dtype='double')   # Levy alpha-stable parameter
        
        # Individual parameters per cond and accumulator: p, x_acc
        p_cond = params[:, :n_p].reshape((n_sim, self.n_accumulators, self.n_conditions))
        x_acc = np.zeros_like(p_cond, dtype='double')
        
        # initialize sim_data array
        sim_data = np.empty((n_sim, n_obs), dtype=np.float32)
        
        # indexing along n_obs dimension
        start_idx = end_idx = 0
        for c in range(self.n_conditions):
            start_idx, end_idx = end_idx, end_idx + n_obs_cond[c]  # increment by current n_obs_cond
            cond_data = batch_generate_levy_lca(n_obs_cond[c], 
                                                p_cond[:, :, c], x_acc[:, :, c],  # individual 
                                                kappa, beta, a, ndt, alpha)       # shared
            sim_data[:, start_idx:end_idx] = np.array(cond_data, dtype=np.float32)
        
        cond_row = np.concatenate([n_obs_cond[c] * [c] for c in range(self.n_conditions)])
        cond_arr = np.stack(n_sim * [cond_row])
        sim_data = np.stack((sim_data, cond_arr), axis=-1)
        
        return sim_data

In [5]:
# levy_lca_simulator() debugger
prior = LCAPrior(n_conditions=2, n_accumulators=2)
simulator = LCASimulator(n_conditions=2, n_accumulators=2)
p = prior(6)
x = simulator(p, 10)
print(p[[0, 2,4]])
print(x[[0, 2,4]])
print(x.shape)

[[1.63436814 0.54493887 1.96567173 0.19449826 0.23269452 0.98280085
  0.43122805 0.39303889]
 [1.23874067 1.05948886 0.47280855 0.77688269 0.63313364 0.80720777
  0.23222514 0.27481739]
 [1.94853453 0.45842388 0.60293779 0.92154032 0.08411581 0.76959913
  0.68616878 0.10634428]]
[[[ 0.41103888  0.        ]
  [ 0.42403889  0.        ]
  [-0.42903888  0.        ]
  [ 0.45603889  0.        ]
  [-0.40503889  0.        ]
  [ 0.60703892  1.        ]
  [-0.40803888  1.        ]
  [-0.41303888  1.        ]
  [-0.41003889  1.        ]
  [ 0.41203889  1.        ]]

 [[ 0.70781738  0.        ]
  [ 0.51881737  0.        ]
  [-0.39481738  0.        ]
  [-0.37081739  0.        ]
  [ 0.53681737  0.        ]
  [-0.44581738  1.        ]
  [ 0.39981738  1.        ]
  [-0.4258174   1.        ]
  [ 0.68181741  1.        ]
  [ 0.55881739  1.        ]]

 [[ 0.11234428  0.        ]
  [-0.11834428  0.        ]
  [ 0.13234428  0.        ]
  [-0.10934428  0.        ]
  [-0.11434428  0.        ]
  [-0.10934428  

In [6]:
D = 8 # 

summary_meta = {
    'n_dense_s1': 2,
    'n_dense_s2': 2,
    'n_dense_s3': 2,
    'n_equiv':    2,
    'dense_s1_args': {'activation': 'relu', 'units': 128},
    'dense_s2_args': {'activation': 'relu', 'units': 128},
    'dense_s3_args': {'activation': 'relu', 'units': 128},
}

summary_net = InvariantNetwork(summary_meta)  # Output size 128


inference_meta = {
    'n_coupling_layers': 4,
    's_args': {
        'units': [64, 64, 64],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    't_args': {
        'units': [64, 64, 64],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    'n_params': D,
    'alpha': 1.9,
    'permute': True
}

inference_net = InvertibleNetwork({'n_params': D})

amortizer = SingleModelAmortizer(inference_net, summary_net)

2021-05-25 19:22:19.776628: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-25 19:22:19.776905: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
generative_model = GenerativeModel(prior=prior, simulator=simulator)

In [8]:
p, x = generative_model(32, 250)
print(p.shape, x.shape)

(32, 8) (32, 250, 2)


In [9]:
trainer = ParameterEstimationTrainer(amortizer,
                      generative_model,
                      learning_rate=0.0001,
                     )

In [ ]:
losses = trainer.train_online(epochs=2, iterations_per_epoch=100, batch_size=16,
                                            n_obs=np.random.randint(100, 201))

In [ ]:
losses = trainer.train_rounds(epochs=2, rounds=2, sim_per_round=100, batch_size=32, 
                              n_obs=np.random.randint(100, 501))

In [ ]:
# Validate (quick and dirty)
p, x = trainer._forward_inference(100, 100)
param_samples = trainer.network.sample(x, n_samples=1000)
param_means = param_samples.mean(axis=0)
true_vs_estimated(p, param_means, ['p11', 'p12', 'p21', 'p22', 'a', 'beta', 'kappa', 'ndt'], figsize=(20,8))